# SOC Triage Lab – Notebook

This notebook is used to manually analyze SOC alerts from the `alerts/` folder.

**Workflow:**
1. Set the alert file you want to analyze.
2. Load and inspect the alert.
3. Use this notebook together with the corresponding report in `reports/`.

# Alert 001 — SSH Brute-Force Attempt with Successful Login

**Category:** Authentication attack / Brute-force  
**Source:** EDR  
**Goal:** Validate whether repeated failed SSH logins followed by a success indicate compromise.  
**What to check:** Source IP, user account, event frequency, successful login timing, device identity.


In [3]:
from pathlib import Path
import json
from pprint import pprint

# Set which alert to load
ALERT_FILE = Path("../alerts/alert_001_ssh_bruteforce.json")

with ALERT_FILE.open("r", encoding="utf-8") as f:
    alert = json.load(f)

pprint(alert)

{'cloud_provider': 'Azure',
 'dst_host': 'webserver-01.example.internal',
 'event_count': 120,
 'id': 'ALERT-001',
 'log_snippet': ['Jan 24 12:15:03 webserver-01 sshd[1024]: Failed password for '
                 'root from 203.0.113.45 port 51234 ssh2',
                 'Jan 24 12:17:41 webserver-01 sshd[1056]: Failed password for '
                 'root from 203.0.113.45 port 51302 ssh2',
                 'Jan 24 12:19:22 webserver-01 sshd[1077]: Failed password for '
                 'root from 203.0.113.45 port 51388 ssh2',
                 'Jan 24 12:24:55 webserver-01 sshd[1099]: Accepted password '
                 'for root from 203.0.113.45 port 51410 ssh2',
                 'Jan 24 12:29:57 webserver-01 sshd[1113]: Session opened for '
                 'user root by (uid=0)'],
 'product': 'Azure VM / Linux auth logs',
 'severity': 'Medium',
 'source': 'SIEM',
 'src_ip': '203.0.113.45',
 'timestamps': {'detection_time': '2025-12-06T12:30:10Z',
                'first_seen': '2

## Triage Notes (ALERT-001)

**Short summary:**  
- 120 failed SSH attempts from a single external IP (`203.0.113.45`) targeting `root` on `webserver-01`
- Followed by a succesful root login.
- Strong indication of brute-force compromise.

**Key indicators:**  
- User / account:  root
- Source IP:  203.0.113.45 (external)
- Destination host:  webserver-01 (Azure VM)
- Time window:  12:15 -> 12:29 UTC
- Event count:  ~120 failed logins + 1 success

**Initial assessment:**  
- [ ] False Positive  
- [ ] Benign but risky  
- [x] True Positive  
- [ ] Needs more information  

**Severity:**  
- [ ] Low  
- [ ] Medium  
- [ ] High  
- [x] Critical  

**Decision (L1):**  
- [ ] Close  
- [x] Escalate to L2 / IR  
- [ ] Create follow-up task  

**Notes for L2:**  
- Confirmed unauthorized root login after repeated failures.
- Recommended immediate VM isolation, credential reset, and forensic review.

# Alert 002 — Impossible Travel Sign-In

**Category:** Identity anomaly / Account compromise  
**Source:** SIEM (Azure AD Sign-In Logs)  
**Goal:** Determine whether two sign-ins from different continents in minutes indicate stolen credentials.  
**What to check:** Locations, IPs, MFA use, device differences, time gap, risk flags.


In [4]:
from pathlib import Path
import json
from pprint import pprint

# Set which alert to load
ALERT_FILE = Path("../alerts/alert_002_impossible_travel.json")

with ALERT_FILE.open("r", encoding="utf-8") as f:
    alert = json.load(f)

pprint(alert)

{'correlation': {'estimated_distance_km': 6200,
                 'impossible_travel_flag': True,
                 'time_diff_minutes': 12},
 'id': 'ALERT-002',
 'log_snippet': ['Sign-in success: alice.werner@example.com from 198.51.100.23 '
                 '(Frankfurt, DE), device WIN10-LAPTOP-01, MFA required.',
                 'Sign-in success: alice.werner@example.com from 203.0.113.88 '
                 '(New York, US), device BROWSER-EDGE, MFA not required.',
                 'Risk event: Impossible travel detected for user '
                 'alice.werner@example.com (12 minutes between Germany and '
                 'United States).'],
 'product': 'Azure AD Sign-In Logs',
 'risk_info': {'risk_event_types': ['impossibleTravel'], 'risk_level': 'high'},
 'severity': 'High',
 'sign_in_events': [{'auth_method': 'Password + MFA',
                     'client_app': 'Outlook',
                     'device': 'WIN10-LAPTOP-01',
                     'event_id': 'aad-evt-1001',
          

## Triage Notes (ALERT-002)

**Short summary:**
- Same user (`alice.werner@example.com`) signs in from Germany and 12 minutes later from the United States.
- Second sign-in uses a different device and no MFA, unlike the first.
- Physical travel is impossible -> strong indication of credential theft or session hijacking.

**Key indicators:**
- User / account: alice.werner@example.com
- First location: Frankfurt, Germany (198.51.100.23) - MFA required
- Second location: New York, USA (203.0.113.88) - Password only
- Time window: 08:12 -> 08:24 UTC (12 minutes)
- Discance: ~6200 km (impossible)
- Risk flags: `impossibleTravel = true`, `risk_level = high`

**Initial assessment:**
- [ ] False Positive  
- [ ] Benign but risky  
- [x] True Positive  
- [ ] Needs more information  

**Severity:**  
- [ ] Low  
- [ ] Medium  
- [x] High  
- [ ] Critical  

**Decision (L1):**  
- [ ] Close  
- [x] Escalate to L2 / IR  
- [ ] Create follow-up task

**Notes for L2:**
- Likely credential compromise; user should be contacted to confirm legitimacy.
- Recommend forced sign-out across sessions and immediate password reset.
- Check for additional risky sign-ins, mailbox access, file actions, or MFA manipulation.
- Investigate wheter other accounts show similar impossible travel patterns.

# Alert 003 — Suspicious PowerShell Execution (Encoded Command)

**Category:** Endpoint / Execution / Malware Delivery  
**Source:** EDR (Microsoft Defender for Endpoint)  
**Goal:** Determine whether a PowerShell command using encoded payload and stealth flags indicates malicious behavior.  
**What to check:** Command line, encoding, process parent, user, outbound connections, domain reputation.

In [1]:
from pathlib import Path
import json
from pprint import pprint

ALERT_FILE = Path("../alerts/alert_003_suspicious_powershell.json")

with ALERT_FILE.open("r", encoding="utf-8") as f:
    alert = json.load(f)

pprint(alert)

{'device': 'WIN-CL-023',
 'id': 'ALERT-003',
 'log_snippet': ['PowerShell launched with suspicious flags (-NoP, '
                 '-EncodedCommand)',
                 'Base64 payload detected',
                 'Outbound HTTPS connection to 185.199.110.153',
                 "Detection: 'SuspiciousPowerShellCommand'"],
 'network_activity': {'domain': 'malicious-test-payload.example',
                      'dst_ip': '185.199.110.153',
                      'dst_port': 443,
                      'outbound_connection': True},
 'process_info': {'command_line': 'powershell.exe -NoP -NonI -W Hidden '
                                  '-EncodedCommand '
                                  'SQBtAHAAcgBvAHIAdAAgACcAaAB0AHQAcAAnADsA',
                  'decoded_command_preview': "Import 'http';",
                  'is_encoded': True,
                  'parent_process': 'explorer.exe',
                  'process_name': 'powershell.exe'},
 'product': 'Microsoft Defender for Endpoint',
 'risk_signal

## Triage Notes (ALERT-003)

**Short summary:**
- PowerShell launched with stealth flags and a Base64-encoded payload.
- Non-admin user executed encoded command.
- Outbound HTTPS request to suspicious domain.
- Strong indication of malicious script execution or initial payload delivery.

**Key indicators:**
- User / account: marcel.schmidt (non-admin)
- Process: powershell.exe (from explorer.exe)
- Command flags: -NoP, -NonI, -W Hidden, -EncodedCommand
- Base64 payload present
- Outbound connection: 185.199.110.153:443
- Domain: malicious-test-payload.example

**Initial assessment:**
- [ ] False Positive
- [ ] Benign but risky
- [x] True Positive
- [ ] Needs more information

**Decision (L1):**
- [ ] Close
- [x] Escalate to L2 / IR
- [ ] Create follow-up task

**Notes for L2:**
- Review decoded payload and full command.
- Check for persistence mechanisms (schedules tasks, registry, startup folder).
- Investigate lateral movement attempts.
- Collect network indicators (domain + IP) for blocking.
- Isolate endpoint if signs of compromise persist.

# Alert 004 — Azure Privilege Escalation (Unauthorized GA Role Assignment)

**Category:** Cloud Identity / Privilege Escalation  
**Source:** SIEM (Azure AD / Entra ID)  
**Goal:** Identify whether the role assignment to a highly privileged role indicates unauthorized elevation.  
**What to check:** Actor identity, PIM usage, justification, sign-in risk, method (API/portal), privilege level.

In [2]:
from pathlib import Path
import json
from pprint import pprint

ALERT_FILE = Path("../alerts/alert_004_azure_privilege_escalation.json")

with ALERT_FILE.open("r", encoding="utf-8") as f:
    alert = json.load(f)

pprint(alert)

{'alerts_context': {'previous_privileged_actions': [],
                    'recent_failed_logins': 5,
                    'recent_successful_signins': 1,
                    'related_risk_events': ['unfamiliarSignInProperties',
                                            'tokenAnomaly']},
 'escalation_details': {'assigned_role': 'Global Administrator',
                        'assignment_type': 'Permanent',
                        'method': 'RoleManagement.CreateRoleAssignment',
                        'privileged_role_id': '62e90394-69f5-4237-9190-012177145e10',
                        'target_user_upn': 'lukas.meier@example.com'},
 'id': 'ALERT-004',
 'log_snippet': ['Actor: lukas.meier@example.com',
                 'Action: RoleManagement.CreateRoleAssignment',
                 'Assigned role: Global Administrator (permanent)',
                 'Justification: null',
                 'Method: MS Graph API',
                 'Role assignment outside PIM approval workflow'],
 'pim_co

## Triage Notes (ALERT-004)
**Short summary:**
- Non-admin user (`lukas.meier@example.com`) assigned himself the **Global Administratir** role permamently.
- Action was performed through MS Graph API, outside the PIM workflow, and without justification.
- Recent sign-in risk events suggest potential credential compromise or token abuse.

**Key indicators:**
- Actor: lukas.meier@example.com (not an admin)
- Assigned role: Flobal Administrator (permament!)
- Method: RoleManagement.CreateRoleAssignment (MS Grapg API)
- No PIM activation or approval
- Risk events: unfamiliarSignInProperties, tokenAnomaly
- Failed sign-ins: 5 in last timeframe
- No business justification provided

**Initial assessment:**
- [ ] False Positive
- [ ] Benign but risky
- [x] True Positive
- [ ] Needs more information

**Severity:**
- [ ] Low
- [ ] Medium
- [ ] High
- [x] Critical

**Decision (L1):**
- [ ] Close
- [x] Escalate to L2 / IR immediately
- [ ] Create follow-up task

**Notes for L2:**
- Likely unauthorized privilege escalation -> potential tenant-wide compromise.
- Revoke Global Admin assignment immediately.
- Force-reset user credentials and invalidate refresh tokens.
- Review MS Graph API activity and app registrations.
- Investigate wheter attacker escalated privileges to persistence roles (e.g.: App Admin, Privileged Auth Admin).

# Alert 005 - Suspicious OAuth App Consent (High-Privilege Graph Scopes)

**Category:** Cloud Identity / OAuth Abuse / Persistence
**Source:** SIEM (Azure AD / Entra ID)
**Goal:** Determine wheter user-granted OAuth consent enables unauthorized, persistent access to tenant data.
**What to check:** App age, publisher verification, granted scopes, admin approval, user role, token usage.

In [1]:
from pathlib import Path
import json
from pprint import pprint

ALERT_FILE = Path("../alerts/alert_005_suspicious_oauth_consent.json")

with ALERT_FILE.open("r", encoding="utf-8") as f:
    alert = json.load(f)

pprint(alert)

{'application': {'app_id': '9f3b9e8a-1d7a-4b4f-8e2a-2f9e7a9a1234',
                 'consent_type': 'User consent',
                 'created_time': '2025-12-13T09:58:02Z',
                 'display_name': 'Invoice Viewer',
                 'publisher_verified': False},
 'context': {'client_app': 'Browser',
             'consent_via': 'OAuth',
             'ip_address': '185.212.44.91',
             'location': 'Unknown',
             'risk_signals': ['Unverified publisher',
                              'High-privilege Graph scopes',
                              'Newly created app',
                              'No admin approval']},
 'id': 'ALERT-005',
 'log_snippet': ['User anna.kowalska@example.com granted OAuth consent to app '
                 "'Invoice Viewer'",
                 'Granted scopes: Mail.Read, Files.Read.All, User.Read.All',
                 'Publisher verified: false',
                 'Admin approval: not required / not granted',
                 'Graph API acce

## Triage Notes (ALERT-005)

**Short summary:**
- Non-admin user granted OAuth consent to a newly created, unverified application.
- App received highßprivilege Microsoft Graph scopes enabling mailbox and file access.
- No admin approval or business justification present ß: potential persistence via OAuth tokens.

**Key indicators:**
- User: anna.kowalska@example.com (non-admin)
- App: "Invoice Viewer" (unverified, newly created)
- Consent type: User consent (browser)
- Scopes: Mail.Read, Files.Read.All, User.Read.All
- Admin approval: required but not granted
- IP / location: 185.212.44.91 / Unknown

**Initial assessment:**
- [ ] False Positive
- [ ] Benign but risky
- [x] True Positive
- [ ] Needs more information

**Severity:**
- [ ] Low
- [ ] Medium
- [x] High
- [ ] Critical

**Decision (L1):**
- [ ] Close
- [x] Escalate to L2 / IR
- [ ] Create follow-up task

**Notes for L2:**
- OAuth tokens may allow persistent access without re-authentication.
- Revoke app consent and invalidate tokens.
- Review mailbox/file access performed by the app.
- Check for additional OAuth apps with similar scopes.
- Consider tightening user-consent policies.